In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from gan3d.utils import loadObjects
from gan3d.progressive_training import training

# Parameters

In [4]:
DATA_PATH = '/home/ecbm4040/3DShapeNets/volumetric_data/chair/30/train/'
PROGRESS = [
    #{'len_arr': 2, 'pixel_shape': 16, 'pro_growth': 1},
    #{'len_arr': 3, 'pixel_shape': 32, 'pro_growth': 2},
    {'len_arr': 4, 'pixel_shape': 64, 'pro_growth': 3},
           ]
G_LAYERS = [{'no_filters': 512, 'padding': 'valid', 'strides': (1, 1, 1)}
            ,{'no_filters': 256, 'padding': 'same', 'strides': (2, 2, 2)}
            ,{'no_filters': 128, 'padding': 'same', 'strides': (2, 2, 2)}
            ,{'no_filters': 64, 'padding': 'same', 'strides': (2, 2, 2)}
           ]
D_LAYERS = [{'no_filters': 64, 'padding': 'same'}
            ,{'no_filters': 128, 'padding': 'same'}
            ,{'no_filters': 256, 'padding': 'same'}
            ,{'no_filters': 512, 'padding': 'same'}
           ]
BATCH_SIZE = 64
NO_BATCHES = 15001
G_LR = 0.0025
D_LR = 0.00001
BETA = 0.5
D_TRESH = 0.8
G_TRESH = 0.2
MODEL_NAME = 'gan3d_refactored_progressive_final_64px_15001'

# Training

In [ ]:
pre_trained_model = None

for progress_step in PROGRESS:
    pixel_shape = progress_step['pixel_shape']
    len_arr = progress_step['len_arr']
    pro_growth = progress_step['pro_growth']
    # load data
    volumes = loadObjects(DATA_PATH, cube_len=pixel_shape, num_obj=7000)
    volumes = volumes[...,np.newaxis].astype(np.float)
    training(volumes,
             d_layers = D_LAYERS[0:len_arr],
             g_layers = G_LAYERS[0:len_arr],
             batch_size = BATCH_SIZE,
             no_batches = NO_BATCHES,
             pixel_shape = pixel_shape,
             d_lr = D_LR,
             g_lr = G_LR,
             beta = BETA,
             d_thresh = D_TRESH,
             g_thresh = G_TRESH,
             model_name = MODEL_NAME + '_' + str(pro_growth),
             pro_growth = pro_growth,
             pre_trained_model = pre_trained_model,
             use_timestamp = False,
             no_models = 1
            )
    pre_trained_model = MODEL_NAME + '_' + str(pro_growth)